# Test embedalign with SentEval 

This notebook will allow you to test EmbedAlign using SentEval. In particular, this also works on **CPUs** :D

* Dependencies:
    * Python 3.5 with NumPy/SciPy
    * Pytorch 
    * Tensorflow 1.5.0  (for CPUs or GPUs depending on how you plan to run it)
        * For example in MacOS: 
        ```
        pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.5.0-py3-none-any.whl
        ```
    * scikit-learn>=0.18.0
    * dill>=0.2.7.1


* Install `dgm4nlp` by following the instructions [here](https://github.com/uva-slpl/dgm4nlp), we highly recommend the use of `virtualenv`.

In the same `virtualenv`, do the following:

* Clone repo from FAIR github
```
    git clone https://github.com/facebookresearch/SentEval.git
    cd SentEval/
```

* Install senteval
```
    python setup.py install
```

* Download datasets (it takes some time...)
    * these are downstream tasks
    * new Senteval also has probing tasks (https://github.com/facebookresearch/SentEval/tree/master/data/probing) for evaluating linguistic properties of your embeddings. 

```
    cd data/downstream/
    ./get_transfer_data.bash
```

* Download [pretained embedlaign model](https://surfdrive.surf.nl/files/index.php/s/9M4h5zqmYETSmf3)


* The following code evaluates embedalign pretrained embeddings on en-fr Europarl on different NLP downstream tasks.



In [1]:
from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
#import data 
# data.py is part of Senteval and it is used for loading word2vec style files
import senteval
import tensorflow as tf
import logging
from collections import defaultdict
import dill
import dgm4nlp
import pickle

c:\users\tomsc\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class dotdict(dict):
    """ dot.notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

class EmbeddingExtractor:
    """
    This will compute a forward pass with the inference model of EmbedAlign and 
        give you the variational mean for each L1 word in the batch.
        
    Note that this takes monolingual L1 sentences only (at this point we have a traiend EmbedAlign model
        which dispenses with L2 sentences).    
        
    You don't really want to touch anything in this class.
    """

    def __init__(self, graph_file, ckpt_path, config=None):        
        g1 = tf.Graph()
        self.meta_graph = graph_file
        self.ckpt_path = ckpt_path
        
        self.softmax_approximation = 'botev-batch' #default
        with g1.as_default():
            self.sess = tf.Session(config=config, graph=g1)
            # load architecture computational graph
            self.new_saver = tf.train.import_meta_graph(self.meta_graph)
            # restore checkpoint
            self.new_saver.restore(self.sess, self.ckpt_path) #tf.train.latest_checkpoint(
            self.graph = g1  #tf.get_default_graph()
            # retrieve input variable
            self.x = self.graph.get_tensor_by_name("X:0")
            # retrieve training switch variable (True:trianing, False:Test)
            self.training_phase = self.graph.get_tensor_by_name("training_phase:0")
            #self.keep_prob = self.graph.get_tensor_by_name("keep_prob:0")

    def get_z_embedding_batch(self, x_batch):
        """
        :param x_batch: is np array of shape [batch_size, longest_sentence] containing the unique ids of words
        
        :returns: [batch_size, longest_sentence, z_dim]        
        """
        # Retrieve embeddings from latent variable Z
        # we can sempale several n_samples, default 1
        try:
            z_mean = self.graph.get_tensor_by_name("z:0")
            
            feed_dict = {
                self.x: x_batch,
                self.training_phase: False,
                #self.keep_prob: 1.

            }
            z_rep_values = self.sess.run(z_mean, feed_dict=feed_dict) 
        except:
            raise ValueError('tensor Z not in graph!')
        return z_rep_values

This is how you interface with SentEval. The only think you need to change are the paths to trained models in the main block at the end.

In [3]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#



# Set PATHs
# path to senteval
#PATH_TO_SENTEVAL = '../'



# import SentEval
#sys.path.insert(0, PATH_TO_SENTEVAL)

# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': '',
                   'usepytorch': False,
                   'kfold': 10,
                   'ckpt_path': '',
                   'tok_path': '',
                   'extractor': None,
                   'tks1': None}
# made dictionary a dotdict
params_senteval = dotdict(params_senteval)
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}



def prepare(params, samples):
    """
    In this example we are going to load a tensorflow model, 
    we open a dictionary with the indices of tokens and the computation graph
    """
    params.extractor = EmbeddingExtractor(
        graph_file='%s.meta'%(params.ckpt_path),
        ckpt_path=params.ckpt_path,
        config=None #run in cpu
    )

    # load tokenizer from training
    params.tks1 = dill.load(open(params.tok_path, 'rb'))
    return

def batcher(params, batch):
    """
    At this point batch is a python list containing sentences. Each sentence is a list of tokens (each token a string).
    The code below will take care of converting this to unique ids that EmbedAlign can understand.
    
    This function should return a single vector representation per sentence in the batch.
    In this example we use the average of word embeddings (as predicted by EmbedAlign) as a sentence representation.
    
    In this method you can do mini-batching or you can process sentences 1 at a time (batches of size 1).
    We choose to do it 1 sentence at a time to avoid having to deal with masking. 
    
    This should not be too slow, and it also saves memory.
    """
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []
    for sent in batch:
        # Here is where dgm4nlp converts strings to unique ids respecting the vocabulary
        # of the pre-trained EmbedAlign model
        # from tokens ot ids position 0 is en
        x1 = params.tks1[0].to_sequences([(' '.join(sent))])
        
        # extract word embeddings in context for a sentence
        # [1, sentence_length, z_dim]
        z_batch1 = params.extractor.get_z_embedding_batch(x_batch=x1)
        # sentence vector is the mean of word embeddings in context
        # [1, z_dim]
        sent_vec = np.mean(z_batch1, axis=1)
        # check if there is any NaN in vector (they appear sometimes when there's padding)
        if np.isnan(sent_vec.sum()):
            sent_vec = np.nan_to_num(sent_vec)        
        embeddings.append(sent_vec)
    embeddings = np.vstack(embeddings)
    return embeddings

def load_pickled_data(filename):
    """
    Load pickled adata
    """
    with open(filename, 'rb') as handle:
        data = pickle.load(handle)
    return data

probs = load_pickled_data('word_probabilities.pickle')

def weighted_batcher(params, batch):
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []
    a = 0.01
    average_prob = 1/(len(probs.keys()))
    
    for sent in batch:
        x1 = params.tks1[0].to_sequences([(' '.join(sent))])
        
        # extract word embeddings in context for a sentence
        # [1, sentence_length, z_dim]
        z_batch1 = params.extractor.get_z_embedding_batch(x_batch=x1)
        
        # get weighted vectors before taking the mean.
        z_batch1[:,0,:] = z_batch1[:,0,:] * average_prob #they added a begin of sentence mark 
        
        for i in range(1, z_batch1.shape[1]):
            try:
                word = sent[i-1] #skip the first dim of z_batch1
            except:
                print(sent, x1, z_batch1.shape[1])
                
            p_w = probs.get(word, average_prob)
            scale = a / (a + p_w) # we can tune this a
            z_batch1[:,i,:] = z_batch1[:,i,:] * scale 
        
        # sentence vector is the mean of word embeddings in context
        # [1, z_dim]
        sent_vec = np.mean(z_batch1, axis=1)
        
        if np.isnan(sent_vec.sum()):
            sent_vec = np.nan_to_num(sent_vec)        
        embeddings.append(sent_vec)
    embeddings = np.vstack(embeddings)
    return embeddings
    

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    # define paths
    # path to senteval data
    # note senteval adds downstream into the path
    params_senteval.task_path = 'C:/Users/TomSc/stack/AI UVA/ULL/lab3/SentEval/data' 
    # path to computation graph
    # we use best model on validation AER
    # TODO: you have to point to valid paths! Use the pre-trained model linked from the top of this notebook.
    params_senteval.ckpt_path = 'C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt'
    # path to tokenizer with ids of trained Europarl data
    # out dictionary id depends on dill for pickle
    params_senteval.tok_path = 'C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/tokenizer.pickle'
    # we use 10 fold cross validation
    params_senteval.kfold = 10
    se = senteval.engine.SE(params_senteval, weighted_batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks = ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                      'MRPC', 'SICKEntailment', 'STS14']
    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    print(results)

2018-05-30 16:41:14,034 : ***** Transfer task : MR *****




INFO:tensorflow:Restoring parameters from C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-30 16:41:17,771 : Restoring parameters from C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-30 16:41:18,718 : Generating sentence embeddings
2018-05-30 16:46:02,325 : Generated sentence embeddings
2018-05-30 16:46:02,326 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-30 16:46:11,744 : Best param found at split 1: l2reg = 4                 with score 64.44
2018-05-30 16:46:21,341 : Best param found at split 2: l2reg = 4                 with score 64.67
2018-05-30 16:46:31,047 : Best param found at split 3: l2reg = 8                 with score 64.31
2018-05-30 16:46:40,616 : Best param found at split 4: l2reg = 8                 with score 64.83
2018-05-30 16:46:50,469 : Best param found at split 5: l2reg = 8                 with score 64.93
2018-05-30 16:47:00,331 : Best param found at split 6: l2reg = 8                 with score 64.49
2018-05-30 16:47:10,164 : Best param found at split 7: l2reg = 8 

INFO:tensorflow:Restoring parameters from C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-30 16:47:41,358 : Restoring parameters from C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-30 16:47:41,895 : Generating sentence embeddings
2018-05-30 16:49:10,575 : Generated sentence embeddings
2018-05-30 16:49:10,576 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-30 16:49:12,255 : Best param found at split 1: l2reg = 8                 with score 70.89
2018-05-30 16:49:14,103 : Best param found at split 2: l2reg = 4                 with score 69.86
2018-05-30 16:49:15,807 : Best param found at split 3: l2reg = 4                 with score 70.27
2018-05-30 16:49:17,485 : Best param found at split 4: l2reg = 8                 with score 70.39
2018-05-30 16:49:19,186 : Best param found at split 5: l2reg = 4                 with score 70.27
2018-05-30 16:49:20,953 : Best param found at split 6: l2reg = 4                 with score 70.68
2018-05-30 16:49:22,588 : Best param found at split 7: l2reg = 4 

...

INFO:tensorflow:Restoring parameters from C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-30 18:05:12,044 : Restoring parameters from C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-30 18:05:12,457 : Computing embedding for test
2018-05-30 18:11:01,324 : Computed test embeddings
2018-05-30 18:11:01,324 : Computing embedding for train
2018-05-30 18:16:18,090 : Computed train embeddings
2018-05-30 18:16:18,090 : Computing embedding for dev
2018-05-30 18:17:00,184 : Computed dev embeddings
2018-05-30 18:17:00,195 : Training sklearn-LogReg with standard validation..
2018-05-30 18:17:01,670 : [('reg:0.25', 70.6), ('reg:0.5', 71.6), ('reg:1', 72.6), ('reg:2', 71.6), ('reg:4', 71.0), ('reg:8', 72.0)]
2018-05-30 18:17:01,670 : Validation : best param found is reg = 1 with score             72.6
2018-05-30 18:17:01,670 : Evaluating...
2018-05-30 18:17:01,880 : 
Dev acc : 72.6 Test acc : 72.8 for                        SICK entailment

2018-05-30 18:17:01,880 : ***** Transfer task : STS14 *****




INFO:tensorflow:Restoring parameters from C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-30 18:17:02,949 : Restoring parameters from C:/Users/TomSc/stack/AI UVA/ULL/lab3/ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-30 18:17:34,721 : deft-forum : pearson = 0.3802, spearman = 0.3915
2018-05-30 18:18:16,305 : deft-news : pearson = 0.6427, spearman = 0.6089
2018-05-30 18:19:09,544 : headlines : pearson = 0.5933, spearman = 0.5837
2018-05-30 18:20:10,608 : images : pearson = 0.6962, spearman = 0.6912
2018-05-30 18:21:09,349 : OnWN : pearson = 0.7114, spearman = 0.7625
2018-05-30 18:22:17,850 : tweet-news : pearson = 0.6429, spearman = 0.5742
2018-05-30 18:22:17,850 : ALL (weighted average) : Pearson = 0.6258,             Spearman = 0.6180
2018-05-30 18:22:17,850 : ALL (average) : Pearson = 0.6111,             Spearman = 0.6020



{'STS14': {'all': {'pearson': {'mean': 0.6111246036212467, 'wmean': 0.6258046599960156}, 'spearman': {'mean': 0.602015029091083, 'wmean': 0.6180263579891032}}, 'tweet-news': {'nsamples': 750, 'pearson': (0.6429181928873018, 1.0748806924120563e-88), 'spearman': SpearmanrResult(correlation=0.574210132769574, pvalue=5.3101342411806945e-67)}, 'deft-forum': {'nsamples': 450, 'pearson': (0.38021812885999917, 6.313765399784633e-17), 'spearman': SpearmanrResult(correlation=0.39149844529470923, pvalue=6.2072664542966766e-18)}, 'deft-news': {'nsamples': 300, 'pearson': (0.6427284503390039, 2.3467921878549944e-36), 'spearman': SpearmanrResult(correlation=0.6089316774718303, pvalue=7.892633971102246e-32)}, 'headlines': {'nsamples': 750, 'pearson': (0.5932632327096253, 1.6955027703808894e-72), 'spearman': SpearmanrResult(correlation=0.5837432729872343, pvalue=1.0488928405654872e-69)}, 'images': {'nsamples': 750, 'pearson': (0.6961705008674539, 8.845182768766216e-110), 'spearman': SpearmanrResult(co

In [4]:
#pickle results
import pickle
with open('results/embed_align_weighted1e-2.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
import pickle
with open('results/embed_align_weighted1e-2.pickle', 'rb') as handle:
    results = pickle.load(handle)

for k,v in results.items():
    try:
        print(k, v['acc'])
    except:
        print(k, " no accuracy")

MPQA 84.05
MR 64.69
SST2 67.33
SICKEntailment 72.8
MRPC 71.19
SUBJ 79.45
CR 70.22
STS14  no accuracy
TREC 57.0


In [10]:
print(results['STS14']['all'])

{'pearson': {'mean': 0.6111246036212467, 'wmean': 0.6258046599960156}, 'spearman': {'mean': 0.602015029091083, 'wmean': 0.6180263579891032}}
